In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# 데이터 불러오기
train_data = pd.read_csv('텍스트 요약 학습 데이터.csv')
test_data = pd.read_csv('가사요약.csv')

In [ ]:
# 훈련 데이터와 검증 데이터로 나누기
train_set, val_set = train_test_split(train_data, test_size=0.1, random_state=42)

In [ ]:
# 토큰화를 위한 Tokenizer 설정
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_set['가사 전문'])

In [ ]:
# 훈련 데이터와 검증 데이터의 시퀀스 생성
train_sequences = tokenizer.texts_to_sequences(train_set['가사 전문'])
val_sequences = tokenizer.texts_to_sequences(val_set['가사 전문'])

In [ ]:
# 시퀀스 패딩
max_len = 100  # 패딩 길이
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
val_padded = pad_sequences(val_sequences, maxlen=max_len, padding='post', truncating='post')

In [ ]:
# 레이블 데이터 생성
train_labels = tokenizer.texts_to_sequences(train_set['가사 요약'])
val_labels = tokenizer.texts_to_sequences(val_set['가사 요약'])

In [ ]:
# 시퀀스 패딩
train_labels_padded = pad_sequences(train_labels, maxlen=max_len, padding='post', truncating='post')
val_labels_padded = pad_sequences(val_labels, maxlen=max_len, padding='post', truncating='post')

In [ ]:
# seq2seq 모델 구성
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, return_sequences=True))  # Add return_sequences=True for seq2seq
model.add(Dense(max_words, activation=None))  # Remove softmax activation

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 레이블을 원-핫 인코딩
from tensorflow.keras.utils import to_categorical
train_labels_one_hot = to_categorical(train_labels_padded, num_classes=max_words)
val_labels_one_hot = to_categorical(val_labels_padded, num_classes=max_words)

In [ ]:
# 모델 훈련
model.fit(train_padded, train_labels_one_hot, validation_data=(val_padded, val_labels_one_hot), epochs=10, batch_size=32)

Epoch 1/10
4/4 [==============================] - 6s 810ms/step - loss: 13.8816 - accuracy: 0.0000e+00 - val_loss: 15.6188 - val_accuracy: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 2s 524ms/step - loss: 15.5939 - accuracy: 0.0064 - val_loss: 15.5938 - val_accuracy: 0.0045
Epoch 3/10
4/4 [==============================] - 2s 532ms/step - loss: 15.5635 - accuracy: 0.0313 - val_loss: 15.5707 - val_accuracy: 0.0100
Epoch 4/10
4/4 [==============================] - 2s 556ms/step - loss: 15.5310 - accuracy: 0.0425 - val_loss: 15.5263 - val_accuracy: 0.0191
Epoch 5/10
4/4 [==============================] - 2s 526ms/step - loss: 15.5051 - accuracy: 0.0472 - val_loss: 15.1259 - val_accuracy: 0.0318
Epoch 6/10
4/4 [==============================] - 2s 538ms/step - loss: 15.3695 - accuracy: 0.0512 - val_loss: 15.4737 - val_accuracy: 0.0336
Epoch 7/10
4/4 [==============================] - 2s 547ms/step - loss: 15.4694 - accuracy: 0.0524 - val_loss: 15.4949 - val_accuracy: 0.028

In [ ]:
# 테스트 데이터 전처리
test_sequences = tokenizer.texts_to_sequences(test_data['가사'])
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

In [ ]:
# 예측
predictions = model.predict(test_padded)

4/4 [==============================] - 1s 77ms/step


In [ ]:
# 예측 결과 디코딩
decoded_predictions = []
for prediction in predictions:
    decoded_predictions.append(tokenizer.sequences_to_texts([prediction.argmax(axis=-1)])[0])

In [ ]:
# 결과 출력
test_data['예측 요약'] = decoded_predictions
print(test_data[['예측 요약']])

'''
결과에서 보이는 <OOV>는 Out Of Vocabulary(단어 집합에 없는 단어)의 약자.
이는 모델이 학습 과정에서 보지 못한 단어나 훈련 시에 포함되지 않은 단어를 나타냄.
'''

                                                예측 요약
0   <OOV> 싶은 싶은 싶은 <OOV> <OOV> <OOV> <OOV> <OOV> <...
1   <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV...
2   <OOV> <OOV> 싶은 <OOV> 싶은 <OOV> <OOV> <OOV> <OOV...
3   싶은 <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <...
4   <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV...
..                                                ...
95  <OOV> <OOV> <OOV> 싶은 <OOV> <OOV> <OOV> <OOV> <...
96  싶은 싶은 싶은 싶은 싶은 싶은 싶은 <OOV> <OOV> <OOV> <OOV> 싶...
97  싶은 싶은 싶은 <OOV> 싶은 싶은 <OOV> 싶은 싶은 싶은 <OOV> 싶은 싶...
98  싶은 마음을 싶은 싶은 싶은 싶은 <OOV> <OOV> <OOV> 싶은 싶은 <OO...
99  아름다운 <OOV> <OOV> 싶은 싶은 <OOV> 싶은 싶은 <OOV> 싶은 싶은...

[100 rows x 1 columns]
